# 16.0简介

核心知识点
- 逻辑回归（Logistic Regression）虽名为 “回归”，但本质是有监督分类方法，用于预测观察值属于某一类别的概率。
- 可扩展为多元逻辑回归（Multinomial Logistic Regression），处理多分类问题。
- 核心优势：输出概率值，可解释性强，计算效率高，是工业界最常用的分类算法之一。
- 本章重点：使用 scikit-learn 训练、调优和评估各类逻辑回归分类器。

# 16.1 训练二元分类器

问题描述
- 训练一个简单的二元分类器，将观察值分为两个类别。

In [2]:
# 加载库
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

from notebooks.Chapter15_KNN.ch15_KNN import new_observation

# 加载仅有两个分类的数据
iris = datasets.load_iris()
features = iris.data[:100,:]
target = iris.target[:100]

# 标准化特征
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# 创建一个逻辑回归对象
logistic_regression = LogisticRegression(random_state=0)

# 训练模型
model = logistic_regression.fit(features_standardized, target)

In [3]:
# 创建一个观察值
new_observation = [[.5,.5,.5,.5]]

# 预测分类
model.predict(new_observation)

array([1])

In [4]:
# 查看预测的概率
model.predict_proba(new_observation)

array([[0.17740549, 0.82259451]])

## 核心知识点

1.  **二元分类本质**：逻辑回归是二元分类器，目标向量仅能取两个值（0或1）。

2.  **核心公式（Sigmoid 函数）**：
    $$
    P(y_i = 1 | X) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 x_i)}}
    $$
    - $P(y_i = 1 | X)$：第 $i$ 个观察值属于类别 1 的概率。
    - $\beta_0, \beta_1$：模型需要学习的参数。
    - 输出值被限制在 $[0, 1]$ 之间，若 $P > 0.5$，则预测为类别 1，否则为类别 0。

3.  **预测方法**：
    - `predict()`：直接输出预测类别。
    - `predict_proba()`：输出样本属于每个类别的概率，可用于评估预测置信度。

# 16.2 训练多元分类器

问题描述
处理包含多个类别的数据集，训练多分类器。

In [1]:
# 加载库
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

# 加载数据
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 标准化特征
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# 创建一对多的逻辑回归对象
logistic_regression = LogisticRegression(random_state=0,multi_class='ovr')

# 训练模型
model = logistic_regression.fit(features_standardized, target)

## 核心知识点

## 1. 一对多（One-vs-Rest, OVR）

### 工作原理
- 对每个类别训练一个独立的二元分类器，判断观察值是否属于该类别。
- 假设各类别问题相互独立，实现简单，但概率校准性较差。

### 特点
| 优点 | 缺点 |
|------|------|
| 实现简单，计算效率高 | 类别间独立性假设可能不成立 |
| 可并行训练多个分类器 | 概率校准性较差 |
| 适用于大规模数据 | 可能产生不一致的预测结果 |

## 2. 多元逻辑回归（Multinomial Logistic Regression, MLR）

### Softmax函数公式
$$
P(y_i = k | X) = \frac{e^{a_k^T x_i}}{\sum_{j=1}^K e^{a_j^T x_i}}
$$

### 参数说明

| 符号 | 含义 | 说明 |
|------|------|------|
| $K$ | 类别总数 | 多分类的类别数量 |
| $a_k^T x_i$ | 线性组合 | 第k个类别的决策函数 |
| $P(y_i = k | X)$ | 条件概率 | 样本属于第k个类别的概率 |

### 核心特点
- **直接输出**：同时输出样本属于每个类别的概率
- **概率校准**：概率值经过更好的校准，预测更可靠
- **归一化**：所有类别概率之和为1

## 3. 参数选择

### 两种策略对比

| 策略 | 参数设置 | 适用场景 | 特点 |
|------|----------|----------|------|
| OVR | `multi_class="ovr"` | 1. 类别较少时<br>2. 计算资源有限<br>3. 需要快速实验时 | 1. 训练K个独立分类器<br>2. 概率未归一化<br>3. 默认设置 |
| MLR | `multi_class="multinomial"` | 1. 类别较多时<br>2. 需要准确的概率估计<br>3. 类别间互斥时 | 1. 单一模型处理多分类<br>2. 概率归一化<br>3. 理论更完备 |



# 16.3 通过正则化来减小方差

In [1]:
# 加载库
from sklearn.linear_model import LogisticRegressionCV
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

# 加载数据
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 标准化特征
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# 创建一个决策树分类器对象
logistic_regression = LogisticRegressionCV(penalty='l2',Cs= 10,random_state=0,n_jobs=-1)

# 训练模型
model = logistic_regression.fit(features_standardized, target)

## 核心知识点

1.  **正则化原理**：通过在损失函数中添加惩罚项，惩罚复杂模型，减小方差。
    - **L1 惩罚**：$\alpha \sum_{j=1}^p |\hat{\beta}_j|$，可产生稀疏解（部分参数为 0，实现特征选择）。
    - **L2 惩罚**：$\alpha \sum_{j=1}^p \hat{\beta}_j^2$，惩罚大参数值，防止过拟合。

2.  **scikit-learn 中的实现**：
    - 使用参数 `C` 代替 $\alpha$，其中 $C = \frac{1}{\alpha}$。
    - $C$ 越大，正则化强度越弱；$C$ 越小，正则化强度越强。

3.  **自动调优**：`LogisticRegressionCV` 可通过交叉验证自动搜索最优的 $C$ 值，`cs` 参数指定搜索范围或数量。

# 16.4 在超大数据集上训练分类器

In [2]:
# 加载库
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

# 加载数据
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 标准化特征
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# 创建逻辑回归对象
logistic_regression = LogisticRegression(random_state=0,solver = 'sag')

# 训练模型
model = logistic_regression.fit(features_standardized, target)

## 核心知识点
1.  求解器（solver）：scikit-learn 提供多种训练逻辑回归的算法，solver参数用于指定。
2.  随机平均梯度（Stochastic Average Gradient, SAG）：
    - 适合超大数据集，训练速度显著快于传统梯度下降。
    - 对特征尺度非常敏感，必须先对特征进行标准化。
3.  其他求解器：liblinear（适合小数据集、二元分类）、newton-cg、lbfgs（适合多分类）等，scikit-learn 会根据场景自动推荐。

# 16.5 处理不均衡的分类

问题描述：
在类别分布严重不均衡的数据集上训练分类器，避免模型偏向多数类。

In [4]:
# 加载库
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

# 加载数据
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 移除前40个观察值，使分类严重不均衡
features = features[40:,:]
target = target[40:]

# 创建目标向量，0代表分类为0,1代表除分数0以外的其他的分类
target = np.where((target==0),0,1)

# 标准化特征
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# 创建决策树分类器对象
logistic_regression = LogisticRegression(random_state=0,class_weight='balanced')

# 训练模型
model = logistic_regression.fit(features_standardized, target)

## 核心知识点

1.  **不均衡分类的影响**：模型会倾向于预测多数类，导致稀有类的预测性能很差。

2.  **自动权重调整**：`class_weight="balanced"` 会根据类别频率自动计算权重：
    $$
    w_j = \frac{n}{k n_j}
    $$
    - $w_j$：类别j的权重。
    - $n$：总样本数。
    - $n_j$：类别j的样本数。
    - $k$：类别总数。

3.  **效果**：增大稀有类的权重，减小多数类的权重，使模型在训练时更关注稀有类，纠正数据不均衡问题。

# 第16章 逻辑回归·精简总结
逻辑回归**名为回归、实为分类算法**，核心是预测样本属于某一类别的概率，具备可解释性强、计算高效、适配工业场景的特点，是最常用的分类模型之一，本章围绕其训练、调优、适配场景展开：

1. **二元分类（基础场景）**
    仅处理二分类任务，通过**Sigmoid函数**将线性结果映射到0~1之间，以0.5为分类阈值；训练前必须对特征做标准化，模型支持`predict()`直接输出类别、`predict_proba()`输出分类概率，方便判断预测置信度。

2. **多分类扩展**
    支持两种多分类模式：
    - 一对多（OVR）：为每个类别单独训练二元分类器，实现简单；
    - 多元逻辑回归：通过**Softmax函数**直接输出多类别概率，概率校准更精准，通过`multi_class`参数切换两种模式。

3. **正则化防过拟合**
    用L1、L2正则化约束模型复杂度，降低过拟合风险：L1可生成稀疏参数实现特征选择，L2更常用、能缩小参数值；scikit-learn中用参数`C`控制正则强度（`C=1/α`，C越小、正则越强），可通过`LogisticRegressionCV`结合交叉验证自动优选最优`C`值。

4. **超大数据集优化**
    大数据场景下选用**SAG随机平均梯度求解器**，训练速度远快于传统算法；但SAG对特征尺度极度敏感，**必须先做特征标准化**才能保证效果。

5. **不均衡数据处理**
    针对类别样本数量悬殊的场景，设置`class_weight='balanced'`，模型会自动根据样本数量为稀有类别分配更高权重，避免模型偏向多数类，提升稀有类别的预测效果。